In [ ]:
import numpy as np
import torch
import pandas as pd
import odbo
import os

## Get initial experiment design (temp using 2016 data, change to other experiments later)

In [ ]:
# Load dataset
random_seed = 100
np.random.seed(random_seed)
data_test = pd.read_csv('../datasets/GB1_2016_149361.csv', sep=',')
name_pre, Y_test = np.array(data_test['AACombo']), np.array(data_test['Fitness'])
print(Y_test[0])
shuffle_order = np.arange(len(Y_test))
np.random.shuffle(shuffle_order[1:])
name_pre[1:], Y_test[1:] = name_pre[shuffle_order[1:]], Y_test[shuffle_order[1:]]
name = odbo.utils.code_to_array(name_pre)
if os.path.isfile('sele_experiment_GB1_2016.npy') == True:
    name_sele = np.load('sele_experiment_GB1_2016.npy')
    Y_train = np.load('sele_fitness_GB1_2016.npy')
    print('Selected initial experiments no. is ', len(Y_train))
else:
    # Let each site has 20 AA codes at least show up twice 
    sele_indices = odbo.initialization.initial_design(name, least_occurance=[2,2,2,2],verbose = False,random_state=random_seed)
    # Initial experiments are selected to be name_sele with fitness of Y_sele
    name_sele, Y_train = name[sele_indices, :], Y_test[sele_indices]
    print('Selected initial experiments no. is ', len(sele_indices))
print(Y_train, Y_train.max())

## Transform AA codes to average fitness feature 

In [ ]:
# Using average measurement results as features
threshold = max(0.05, np.mean(Y_train)-2*np.std(Y_train))
print(threshold)
feature_model = odbo.featurization.MaxMeasurement(raw_vars=name_sele, Y=Y_train)
X_train = feature_model.transform(name_sele)
X_test = feature_model.transform(name)

## Adapt search space construction

In [ ]:
# Find the adaptive search space model
labels_train = odbo.prescreening.sp_label(X_train, Y_train, thres=threshold)
pre_model = odbo.prescreening.XGBOD(eval_metric = 'error')
pre_model.fit(X_train, labels_train)
pred_labels = pre_model.predict(X_train)
labels_test = odbo.prescreening.sp_label(X_test, Y_test, thres=threshold)
pred_test_labels = pre_model.predict(X_test)

In [ ]:
# Plot the confusion matrix to check the accuracy of search space prescreening
out_outlier, in_outlier, out_inlier, in_inlier = odbo.plot.plot_cm(labels_test, pred_test_labels, Y_test)
print("Correct ratio: {0:.3%}".format((len(out_outlier)+len(in_inlier))/len(labels_test)))
print("FN ratio: {0:.3%}".format(len(out_inlier)/len(labels_test)))
print("FP ratio: {0:.3%}".format(len(in_outlier)/len(labels_test)))  

## Cluster-BO with batch_size =2 for next best experiment

In [ ]:
ncluster_grid=[3,4,5]
# Pick the Top 40 init experiment from the 384 experiments
X_train_sele, Y_train_sele = torch.tensor(X_train), torch.tensor(Y_train.reshape(len(Y_train),1))
# Only search the space after prescreening
sele_id_test = list(np.array([k for k, x in enumerate(pred_test_labels) if x == 0]))
X_test_sele, Y_test_sele = torch.tensor(X_test[sele_id_test, :]), torch.tensor(Y_test[sele_id_test].reshape(len(sele_id_test),1))
search_name_sele, name_sele_temp = name[sele_id_test, :], name_sele

## Run BO experiment with robust regression or directly gp
l, search_iter = 0, 50
batch_size = 2
gp_method='gp_regression'
failure_count,max_count = 0, 0
while l < search_iter:
    print("Iter: ", l, "Current Max: ", Y_train_sele.max().detach().numpy(), "Actual max: ",Y_test_sele.max().detach().numpy(), X_test_sele[np.argmax(Y_test_sele.detach().numpy()), :])
    X_next, acq_value, next_exp_id = odbo.cluster_bo_design(X=X_train_sele, Y=Y_train_sele, ncluster_grid=ncluster_grid, X_pending=X_test_sele, gp_method=gp_method, batch_size=batch_size)
    ids_keep = list(np.delete(range(X_test_sele.shape[0]), next_exp_id))
    X_train_sele, Y_train_sele = torch.cat([X_train_sele, X_test_sele[next_exp_id, :]]), torch.cat([Y_train_sele, Y_test_sele[next_exp_id]])
    X_test_sele, Y_test_sele, search_name_sele = X_test_sele[ids_keep, :], Y_test_sele[ids_keep], search_name_sele[ids_keep]
    name_sele_temp = np.concatenate((name_sele_temp, search_name_sele[next_exp_id]))
    print("Newly added value: ", Y_train_sele[-batch_size:].detach().numpy(), X_train_sele[-batch_size:, :])
    feature_model1 = odbo.featurization.AvgMeasurement(raw_vars=X_train_sele, Y=Y_train_sele.detach().numpy())
    if Y_train_sele[-batch_size:].detach().numpy().max() <= Y_train_sele[:-batch_size].max():
        failure_count = failure_count + 1
    else:
        failure_count = 0
    if failure_count >= 3 and max_count < 3:
        max_count = max_count + 1
        feature_model1 = odbo.featurization.MaxMeasurement(raw_vars=X_train_sele, Y=Y_train_sele.detach().numpy())
    else:
        max_count = 0
    X_train_sele = torch.tensor(feature_model1.transform(X_train_sele))
    X_test_sele= torch.tensor(feature_model1.transform(X_test_sele))
    l = l + 1




## Cluster-TurBO-m for next best experiment

In [ ]:
ncluster_grid=[3,4,5]
X_train_sele, Y_train_sele = torch.tensor(X_train), torch.tensor(Y_train.reshape(len(Y_train),1))
# Only search the space after prescreening
sele_id_test = []
sele_id_test.extend(in_inlier)
sele_id_test.extend(out_inlier)
search_name_sele, name_sele_temp = name[sele_id_test, :], name_sele
X_test_sele, Y_test_sele = torch.tensor(X_test[sele_id_test, :]), torch.tensor(Y_test[sele_id_test].reshape(len(sele_id_test),1))

# Run BO experiment with robust regression or directly gp
l, search_iter = 0, 50
gp_method='gp_regression'
tr_length = [4.8, 3.2]
batch_size = 1
failure_count = 0

state = odbo.turbo.TurboState(dim=X_train_sele.shape[1], batch_size=batch_size, length=tr_length, n_trust_regions=len(tr_length), failure_tolerance = 10)
state.best_value = Y_train_sele.max()
while l < search_iter:
    print("Iter: ", l, "Current Max: ", Y_train_sele.max().detach().numpy(), 'TR length: ', state.length, 'True max ', Y_test_sele.max(), X_test_sele[np.argmax(Y_test_sele.detach().numpy()), :])
    X_next, acq_value, raw_next_exp_id = odbo.cluster_turbo_design(state=state, X=X_train_sele, Y=Y_train_sele, ncluster_grid=ncluster_grid, X_pending=X_test_sele, n_trust_regions=len(tr_length), batch_size=batch_size, gp_method=gp_method)
    Y_next_m = torch.zeros((len(tr_length), batch_size, 1), device=Y_train_sele.device, dtype=Y_train_sele.dtype)
    next_exp_id = []
    print(raw_next_exp_id.shape)
    for i in range(batch_size):
        next_exp_id_m = raw_next_exp_id[:, i]
        Y_next_m[:, i, 0], idtoadd = Y_test_sele[next_exp_id_m].reshape(len(tr_length)), next_exp_id_m[np.argmax(Y_test_sele[next_exp_id_m])]
        next_exp_id.append(idtoadd)
    X_train_sele, Y_train_sele = torch.cat([X_train_sele, X_test_sele[next_exp_id, :]]), torch.cat([Y_train_sele, Y_test_sele[next_exp_id]])
    ids_keep = list(np.delete(range(X_test_sele.shape[0]), next_exp_id))
    X_test_sele, Y_test_sele, search_name_sele = X_test_sele[ids_keep, :], Y_test_sele[ids_keep], search_name_sele[ids_keep]
    name_sele_temp = np.concatenate((name_sele_temp, search_name_sele[next_exp_id]))
    print("Newly added value: ", Y_train_sele[-batch_size:].detach().numpy())
    state = odbo.turbo.update_state(state=state, Y_next=Y_next_m)
    feature_model1 = odbo.featurization.AvgMeasurement(raw_vars=X_train_sele, Y=Y_train_sele.detach().numpy())
    if Y_train_sele[-batch_size:].detach().numpy().max() <= Y_train_sele[:-batch_size].max():
        failure_count = failure_count + 1
    else:
        failure_count = 0
    if failure_count >= 3 and max_count < 3:
        max_count = max_count + 1
        feature_model1 = odbo.featurization.MaxMeasurement(raw_vars=X_train_sele, Y=Y_train_sele.detach().numpy())
    else:
        max_count = 0
    X_train_sele = torch.tensor(feature_model1.transform(X_train_sele))
    X_test_sele= torch.tensor(feature_model1.transform(X_test_sele))

    l = l + 1

